In [1]:
# Avant de lancer la construction du modèle CNN basique on étudie les données disponibles 
import os 
X_train_cat = os.listdir('clean_dataset/train/cat')
X_val_cat = os.listdir('clean_dataset/validation/cat')
X_test_cat = os.listdir('clean_dataset/test/cat')
print("Nous avons " + str(len(X_train_cat)) + " images d'entrainement de chats.")
print("Nous avons " + str(len(X_val_cat)) + " images de validation de chats.")
print("Nous avons " + str(len(X_test_cat)) + " images de test de chats.")

Nous avons 9987 images d'entrainement de chats.
Nous avons 1248 images de validation de chats.
Nous avons 1248 images de test de chats.


In [2]:
X_train_dog = os.listdir('clean_dataset/train/dog')
X_val_dog = os.listdir('clean_dataset/validation/dog')
X_test_dog = os.listdir('clean_dataset/test/dog')
print("Nous avons " + str(len(X_train_dog)) + " images d'entrainement de chiens.")
print("Nous avons " + str(len(X_val_dog)) + " images de validation de chiens.")
print("Nous avons " + str(len(X_test_dog)) + " images de test de chiens.")

Nous avons 9985 images d'entrainement de chiens.
Nous avons 1248 images de validation de chiens.
Nous avons 1248 images de test de chiens.


In [4]:
# Import des librairies utiles 
import sklearn
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense, Dropout
from keras.utils import to_categorical
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
import seaborn as sns
import numpy as np
import pandas as pd
import pickle

In [6]:
# De même que pour la régression logistique, une étape de preprocessing est nécessaire. 
# Elle permettra de standardiser les dimensions des images et de normaliser les valeurs des pixels. 
# En accord avec l'architecture VGG16, nous utiliserons des images de 224*224*3. 
# Pour cela on instancie la méthode ImageDataGenerator de Keras.preprocessing qui permet de normaliser les valeurs de pixels. 
# On utilise cette méthode car elle permettra dans un autre modèle de réaliser de la data augmentation online.
preproc_train = ImageDataGenerator(rescale=1.0/255.0,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=True)

preproc_test = ImageDataGenerator(rescale=1.0/255.0)

In [7]:
# A l'aide de la fonction flow_from_directory() qui nécessite que la structure des dossiers soit celle présentée dans le rapport, 
# Nous allons pouvoir lire les images directement depuis leurs dossiers respectifs.
# Cette méthode permet de fixer la taille des images avec l'argument target_size
# Elle créera des batch d'images. 
# Un batch est caractérisé par sa taille et définit le nombre d'images à traiter par le modèle 
# avant de mettre à jour les poids du modèle. Une valeur commune de batch est 32 ou 64.
# L'avantage d'utiliser flow_from_directory est qu'il infère les labels en se basant sur le nom du dossier.
train_set = preproc_train.flow_from_directory('clean_dataset/train/',
                                        class_mode = 'binary',
                                        batch_size = 64,
                                        target_size = (224, 224))

validation_set = preproc_test.flow_from_directory('clean_dataset/validation',
                                             class_mode = 'binary',
                                             batch_size = 64,
                                             target_size = (224, 224))

test_set = preproc_test.flow_from_directory('clean_dataset/test',
                                       class_mode = 'binary', 
                                       batch_size = 64, 
                                       target_size = (224, 224))

Found 19972 images belonging to 2 classes.
Found 2496 images belonging to 2 classes.
Found 2496 images belonging to 2 classes.


In [8]:
# Une fois les données préparer on peut définir le modèle basique


def basic_cnnx3(): 
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    
    # On doit compiler le modèle en spécifiant le solver = algorithme utilisé pour minimiser l'erreur de prédiction
    # Les valeurs de learning rate et du momentum sont choisies en accord avec les hyperparamètre de VGG16 
    solver = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=solver, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [9]:
# On définit la fonction plot_perf() qui enregistrera deux graphiques d'accuracy et de loss pour le
# set d'entrainement et de validation.
# On précise que history.history contient les différentes métriques de performances réalisées. 
# Il s'agit de la loss et de l'accuracy

def plot_perf(history): 
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    # On regarde l'évolution en fonction des epochs 
    epochs = range(1, len(acc)+1)
    
    # On plot l'accuracy de l'entrainement et de la validation par epoch
    plt.plot(epochs, acc, 'bo', label='Train_Acc')
    plt.plot(epochs, val_acc, 'b', label="Val_Acc")
    plt.title('Accuracy sur Train et Validation')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    filename1 = "Train_Val_Acc_CNN3DA"
    plt.savefig(filename1 + '_plot.png')
    plt.close()
    
    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Train_Loss')
    plt.plot(epochs, val_loss, 'b', label="Val_Loss" )
    plt.title('Loss sur Train et Validation')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    filename2 = "Train_Val_Loss_CNN3DA"
    plt.savefig(filename2 + '_plot.png')
    plt.close()

In [10]:
# On créer une fonction qui regroupe les précédente et qui permet de lancer toutes les opérations.
def launch_learning(): 
    
    # On définit le modèle
    model = basic_cnnx3()
    
    # On fit le modèle sur les données d'apprentissages et on donne les données de validation
    # history contient alors les valeurs de loss et d'accuracy
    # Le paramètre verbose permet d'afficher le temps passé par epoch, et l'accuracy et la loss
    history = model.fit(train_set, 
                       validation_data=validation_set, 
                       epochs=20, 
                       verbose=1)
    
    # On évalue le modèle sur le jeux de données test
    # Par défaut le paramètre setps est None. L'évaluation prendra en compte tous les batch avant de s'arrêter.
    evaluation = model.evaluate(test_set,
                                batch_size=64,
                                verbose=1)
    print(model.metrics_names, evaluation)
    
    # On enregistre le modèle
    filename="CNN3_DataAugmented.h5"
    model.save(filename)
    
    plot_perf(history)
    

In [11]:
launch_learning()

Epoch 1/20
313/313 [==============================] - 439s 1s/step - loss: 0.6768 - accuracy: 0.5837 - val_loss: 0.6329 - val_accuracy: 0.6434
Epoch 2/20
313/313 [==============================] - 449s 1s/step - loss: 0.6309 - accuracy: 0.6424 - val_loss: 0.6055 - val_accuracy: 0.6663
Epoch 3/20
313/313 [==============================] - 457s 1s/step - loss: 0.5966 - accuracy: 0.6807 - val_loss: 0.5663 - val_accuracy: 0.6987
Epoch 4/20
313/313 [==============================] - 459s 1s/step - loss: 0.5794 - accuracy: 0.6938 - val_loss: 0.5205 - val_accuracy: 0.7484
Epoch 5/20
313/313 [==============================] - 452s 1s/step - loss: 0.5411 - accuracy: 0.7243 - val_loss: 0.5253 - val_accuracy: 0.7300
Epoch 6/20
313/313 [==============================] - 452s 1s/step - loss: 0.5255 - accuracy: 0.7355 - val_loss: 0.4725 - val_accuracy: 0.7760
Epoch 7/20
313/313 [==============================] - 452s 1s/step - loss: 0.5065 - accuracy: 0.7522 - val_loss: 0.4695 - val_accuracy: 0.7720